In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from func_helper import *

## 读入数据并处理特征

In [2]:
data_path = './data/combined_data.csv'
raw_data = pd.read_csv(data_path)
raw_data.head()

,WindNumber,Time,WindSpeed,Power,RotorSpeed,Width,RatedPower,CutInWindSpeed,CutOutWindSpeed,WheelSpeedMin,WheelSpesdMax
0,1,2017/11/1 0:20,4.188333,65.750000,5.779167,99.0,2000,3.0,25,8.33,16.8
1,1,2017/11/1 0:30,4.042500,120.700000,8.636500,99.0,2000,3.0,25,8.33,16.8
2,1,2017/11/1 0:40,4.137368,135.157895,8.668421,99.0,2000,3.0,25,8.33,16.8
3,1,2017/11/1 0:50,4.274737,157.315789,8.727895,99.0,2000,3.0,25,8.33,16.8
4,1,2017/11/1 1:00,4.108947,129.894737,8.656842,99.0,2000,3.0,25,8.33,16.8


## 正式开始处理数据

### 一号

In [ ]:
current_rotor = 1
current_data = raw_data[raw_data['WindNumber']==current_rotor]

process_current_data(current_data)

In [ ]:
is_save=True
line_num = 1
fp = "./figures/vp_{}_{}.png".format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

### 二号

In [ ]:
current_rotor = 2
current_data = raw_data[raw_data['WindNumber']==current_rotor]
# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去除最下面的线
low_idx = current_data[((current_data['Month']==1) | (current_data['Month']==2) | (current_data['Month']==12)) & (current_data['WindSpeed']>9)].index
current_data.loc[low_idx, 'label']=1
print("low line: {}".format(low_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data, max_time=9, max_diff=8, power_step=20)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2.5,up_coef=2.8)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data[current_data['WindSpeed']>8])
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2,up_coef=1)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))


In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 三号

- **三号风机两条曲线数据**

1. 第一条到达额定功率2000曲线
    - 5,6,7月，4月12-31号，8月1-21号，10月29号\[99:], 30号\[:50]
2. 第二条功率1000左右的
    - 1,2,3,9,11,12, 4月1-11号，8月22-31号，10月1-28号,29号\[:99], 30号\[50:]

#### 3.1 三号第一条线

In [ ]:
current_rotor = 3
# 处理第一条线 5,6,7月，4月18-31号，8月1-21号
current_data = raw_data[raw_data['WindNumber']==current_rotor]

oct29_data = current_data[((current_data['Month']==10) & (current_data['Day']==29))][99:]
oct30_data = current_data[((current_data['Month']==10) & (current_data['Day']==30))][:50]

current_data = current_data[(current_data['Month']==5) | (current_data['Month']==6) | (current_data['Month']==7) | ( (current_data['Month']==4) & (current_data['Day']>=12) ) | ( (current_data['Month']==8) & (current_data['Day']<=21) )]
current_data = pd.concat([current_data, oct29_data, oct30_data])

# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))
# 应用横部四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=1.5, up_coef=1.5)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))


vertical_data, vertical_anomaly_idx = vertical_process(current_data, low_coef=2, up_coef=2)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

#### 3.2 三号第二条线

In [ ]:
current_rotor = 3
# 处理第二条线 1,2,3,9,11,12, 4月1-11号，8月22-31号，10月1-28号,29号[:99], 30号[50:]
current_data = raw_data[raw_data['WindNumber']==current_rotor]

oct29_data = current_data[((current_data['Month']==10) & (current_data['Day']==29))][:99]
oct30_data = current_data[((current_data['Month']==10) & (current_data['Day']==30))][50:]

current_data = current_data[(current_data['Month']==1) | (current_data['Month']==2) | (current_data['Month']==3) | (current_data['Month']==9) | (current_data['Month']==11) | (current_data['Month']==12) | ( (current_data['Month']==4) & (current_data['Day']<12) ) | ( (current_data['Month']==8) & (current_data['Day']>21) ) | ( (current_data['Month']==10) & (current_data['Day']<29) )]
current_data = pd.concat([current_data, oct29_data, oct30_data])

# process_current_data(current_data)

In [ ]:
# # 去底部异常
# removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
# current_data.loc[neg_power_idx, 'label']=1
# print("bottom: {}".format(neg_power_idx.shape))
# # 应用横部四分法
# horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=1.7, up_coef=1.9)
# current_data.loc[horizonal_anomaly_idx, 'label']=1
# print("horizonal: {}".format(horizonal_anomaly_idx.shape))

In [ ]:
current_data['label']=1

In [ ]:
is_save=True
line_num = 2
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 全部当作异常点
current_anomaly_idx = current_data.index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 四号

In [ ]:
current_rotor = 4
current_data = raw_data[raw_data['WindNumber']==current_rotor]
# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data, low_coef=4, up_coef=3)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data, low_coef=2, up_coef=2)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 五号

In [ ]:
current_rotor = 5
current_data = raw_data[raw_data['WindNumber']==current_rotor]
# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data, low_coef=5, up_coef=4)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))


In [ ]:
# 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data, low_coef=1.5, up_coef=3)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 六号

1. 第一条到达额定功率2000曲线
    - 9，10月
2. 从下往上第二条功率1000左右的
    - 5月18-31号，6，7，8月
3. 最下面功率800左右的
    - 1，2，3，4，11，12月，5月1-17号

In [ ]:
current_rotor = 6
# 处理第一条线 9，10月
current_data = raw_data[raw_data['WindNumber']==current_rotor]
current_data = current_data[(current_data['Month']==9) | (current_data['Month']==10)]
# process_current_data(current_data)

#### 6.1 六号第一条线

In [ ]:
vertical_data, vertical_anomaly_idx = vertical_process(current_data)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

#### 6.2 六号第二条线

In [ ]:
current_rotor = 6
# 处理第二条线 5月18-31号，6，7，8月
current_data = raw_data[raw_data['WindNumber']==current_rotor]
current_data = current_data[(current_data['Month']==6) | (current_data['Month']==7) | (current_data['Month']==8) | ( (current_data['Month']==5) & (current_data['Day']>=18) ) ]
# process_current_data(current_data)

In [ ]:
# # 去底部异常
# removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
# current_data.loc[neg_power_idx, 'label']=1
# print("bottom: {}".format(neg_power_idx.shape))

# # 横向四分法
# horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2, up_coef=2.5)
# current_data.loc[horizonal_anomaly_idx, 'label']=1
# print("horizonal: {}".format(horizonal_anomaly_idx.shape))

In [ ]:
current_data['label']=1

In [ ]:
is_save=True
line_num = 2
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 全部当作异常
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

#### 6.3 六号第三条曲线

In [ ]:
current_rotor = 6
# 处理第三条线 1，2，3，4，11，12月，5月1-17号
current_data = raw_data[raw_data['WindNumber']==current_rotor]
current_data = current_data[(current_data['Month']==1) | (current_data['Month']==2) | (current_data['Month']==3)  | (current_data['Month']==4) | (current_data['Month']==11) | (current_data['Month']==12) | ( (current_data['Month']==5) & (current_data['Day']<18) ) ]
# process_current_data(current_data)

In [ ]:
# # 去底部异常
# removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
# current_data.loc[neg_power_idx, 'label']=1
# print("bottom: {}".format(neg_power_idx.shape))

# # 去中部异常
# removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
# current_data.loc[mid_anomaly_idx, 'label']=1
# print("mid: {}".format(mid_anomaly_idx.shape))

# # 横向四分法
# horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data, low_coef=2.5, up_coef=3.5)
# current_data.loc[horizonal_anomaly_idx, 'label']=1
# print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data, low_coef=1.5, up_coef=1.5)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
current_data['label']=1

In [ ]:
is_save=True
line_num = 3
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 七号

1. 左边曲线
    - 9月10-17号
2. 右边曲线
    - 除了9月，加上0月1-9号，>=18号

#### 7.1 七号第一条

In [ ]:
current_rotor = 7
# 处理左边，9月10-17
current_data = raw_data[raw_data['WindNumber']==current_rotor]
current_data = current_data[( (current_data['Month']==9) & (current_data['Day']<=17) & (current_data['Day']>=10))]
# process_current_data(current_data)

In [ ]:
# 去底部异常
# removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
# current_data.loc[neg_power_idx, 'label']=1
# print("bottom: {}".format(neg_power_idx.shape))

# # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data, low_coef=1.5, up_coef=1.5)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))

# 全部当作异常
current_data['label']=1

#### 如果加上去掉功率异常的, 会导致下面去掉一条线, 是因为在切入风速以内,有power

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

#### 7.2 七号第二条

In [ ]:
current_rotor = 7
# 处理右边，除了9月，加上9月1-9号，>=18号
current_data = raw_data[raw_data['WindNumber']==current_rotor]

sep_data = current_data[( (current_data['Month']==9) & (current_data['Day']>=18) ) | ((current_data['Month']==9) & (current_data['Day']<10))]

current_data = current_data[( (current_data['Month']==1) | (current_data['Month']==2) | (current_data['Month']==3) | (current_data['Month']==4) | (current_data['Month']==5) | (current_data['Month']==6) | (current_data['Month']==7) | (current_data['Month']==8) | (current_data['Month']==10) | (current_data['Month']==11) | (current_data['Month']==12))]
current_data = pd.concat([current_data, sep_data])

# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data, low_coef=1.5, up_coef=1.7)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 2
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 八号

In [ ]:
current_rotor = 8
current_data = raw_data[raw_data['WindNumber']==current_rotor]
# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data,max_diff=2,max_time=6)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data, low_coef=1.5, up_coef=2.2)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data, up_coef=2.2)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 九号

1. 上边曲线
    - 9，10月，8月>26号
2. 下边曲线
    - 除了9月，10月的月份，其中八月：1-26号

#### 9.1 九号第一条

In [ ]:
current_rotor = 9
# 处理左边，9月，10月，8月>26号
current_data = raw_data[raw_data['WindNumber']==current_rotor]
current_data = current_data[( (current_data['Month']==9) | (current_data['Month']==10) | ((current_data['Month']==8) & (current_data['Day']>26)))]
# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,up_coef=2.6)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

#### 9.2 九号第二条线

In [ ]:
current_rotor = 9
# 处理下边，除了9月，10月的月份，其中八月：1-26号
current_data = raw_data[raw_data['WindNumber']==current_rotor]

current_data = current_data[( (current_data['Month']==1) | (current_data['Month']==2) | (current_data['Month']==3) | (current_data['Month']==4) | (current_data['Month']==5) | (current_data['Month']==6) | (current_data['Month']==7) | (current_data['Month']==11) | (current_data['Month']==12) | ((current_data['Month']==8) & (current_data['Day']<=26)) )]

# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data,max_time=6,max_diff=2)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2,up_coef=2.5)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data,low_coef=2, up_coef=2)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 2
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 十月

1. 左边曲线
    - 12, 1月<=7日，11月>=12日
2. 右边曲线
    - 2, 3, 4, 5, 6, 7, 8, 9, 10，1月>7日，11月<12日

#### 10.1 十号第一条

In [ ]:
current_rotor = 10
# 处理左边，12, 1月<=7日，11月>=12日
current_data = raw_data[raw_data['WindNumber']==current_rotor]
current_data = current_data[( (current_data['Month']==12) | ((current_data['Month']==1)&(current_data['Day']<=7) ) | ((current_data['Month']==11) & (current_data['Day']>=12)))]
# process_current_data(current_data)

In [ ]:
# # 去底部异常
# removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
# current_data.loc[neg_power_idx, 'label']=1
# print("bottom: {}".format(neg_power_idx.shape))

# # 去中部异常
# removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
# current_data.loc[mid_anomaly_idx, 'label']=1
# print("mid: {}".format(mid_anomaly_idx.shape))

# # # 横向四分法
# horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2,up_coef=2)
# current_data.loc[horizonal_anomaly_idx, 'label']=1
# print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data,low_coef=2, up_coef=2)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))

# # # 横向四分法
# horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data)
# current_data.loc[horizonal_anomaly_idx, 'label']=1
# print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))
# # # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
# 全部作为异常
current_data['label']=1
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

#### 10.2 十号第二条

In [ ]:
current_rotor = 10
# 处理右边，2, 3, 4, 5, 6, 7, 8, 9, 10，1月>7日，11月<12日
current_data = raw_data[raw_data['WindNumber']==current_rotor]

current_data = current_data[( ((current_data['Month']==1)&(current_data['Day']>7)) | (current_data['Month']==2) | (current_data['Month']==3) | (current_data['Month']==4) | (current_data['Month']==5) | (current_data['Month']==6) | (current_data['Month']==7) | (current_data['Month']==8) | (current_data['Month']==9) | (current_data['Month']==10)| ((current_data['Month']==11)&(current_data['Day']<12)) )]

# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data,max_time=6,max_diff=2)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# # 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2,up_coef=2.1)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # 纵向四分法
vertical_data, vertical_anomaly_idx = vertical_process(current_data,low_coef=2,up_coef=1.9)
current_data.loc[vertical_anomaly_idx, 'label']=1
print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 2
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 十一号

In [ ]:
current_rotor = 11
current_data = raw_data[raw_data['WindNumber']==current_rotor]
# process_current_data(current_data)

In [ ]:
# 去底部异常
removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
current_data.loc[neg_power_idx, 'label']=1
print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# # 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=2.5,up_coef=3)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data,low_coef=2,up_coef=2)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 1
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

### 十二号

In [ ]:
current_rotor = 12
current_data = raw_data[raw_data['WindNumber']==current_rotor]
# process_current_data(current_data)

In [ ]:
# 去底部异常
# removed_neg_data, neg_power_idx = remove_neg_power(current_data, current_rotor)
# current_data.loc[neg_power_idx, 'label']=1
# print("bottom: {}".format(neg_power_idx.shape))

# 去中部异常
removed_mid_data, mid_anomaly_idx = remove_mid_anomaly(current_data)
current_data.loc[mid_anomaly_idx, 'label']=1
print("mid: {}".format(mid_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=3,up_coef=3)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# 横向四分法
horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=1.5,up_coef=1.5)
current_data.loc[horizonal_anomaly_idx, 'label']=1
print("horizonal: {}".format(horizonal_anomaly_idx.shape))

# # 纵向四分法
# vertical_data, vertical_anomaly_idx = vertical_process(current_data,low_coef=2,up_coef=2)
# current_data.loc[vertical_anomaly_idx, 'label']=1
# print("vertical: {}".format(vertical_anomaly_idx.shape))

In [ ]:
is_save=True
line_num = 2
fp = './figures/vp_{}_{}.png'.format(current_rotor,line_num)
plot_current_data(current_data,rotor_num=current_rotor,save=is_save,file_path=fp)

In [ ]:
# 更新到raw_data
current_anomaly_idx = current_data[current_data['label']==1].index
print("sum of current anomaly data: {}".format(current_anomaly_idx.shape))

raw_data.loc[current_anomaly_idx, 'label']=1
print("sum of raw data anomaly data: {}".format(np.sum(raw_data['label']==1)))

```
sum of current anomaly data: (10565,)
sum of raw data anomaly data: 158009
```

## 保存数据

In [ ]:
raw_data.loc[raw_data['label']==-1, 'label']=0

In [ ]:
# current_data = raw_data
# # 横向四分法
# horizonal_data, horizonal_anomaly_idx = horizonal_process(current_data,low_coef=1.5,up_coef=1.5)
# current_data.loc[horizonal_anomaly_idx, 'label']=1
# print("horizonal: {}".format(horizonal_anomaly_idx.shape))

In [ ]:
file_num = "9_25"
RES_PATH='./submission/submission_{}.csv'.format(file_num)
# print("未分类数：{}".format(np.sum(raw_data['label'] == -1)))
print("异常点数：{}".format(np.sum(raw_data['label'] == 1)))
print("正常点数：{}".format(np.sum(raw_data['label'] == 0)))

result = raw_data[['WindNumber', 'Time', 'label']]
result.to_csv(RES_PATH, index=None)